In [1]:
import yfinance as yf
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error
import numpy as np
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
from pmdarima.arima import auto_arima
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.models import Sequential, load_model
from keras import layers
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import warnings
warnings.filterwarnings("ignore", category=pd.core.common.SettingWithCopyWarning)
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
def LSTM_OS_real(hist_data):
    scaler = MinMaxScaler(feature_range = (0, 1))
    data_scaled = scaler.fit_transform(hist_data)

    X, y = [], []
    window_size = 30
    prediction_size = 10
    for i in range(window_size, len(data_scaled)-prediction_size+1):
        X.append(data_scaled[i-window_size:i])
        y.append(data_scaled[i:i+prediction_size])
    X, y = np.array(X), np.array(y)

    train_split = int(len(X) * 0.8)
    X_train = X[:train_split]
    y_train = y[:train_split]
    X_val = X[train_split:] 
    y_val = y[train_split:] 


    model = Sequential([layers.Input((window_size, 1)),
                    layers.LSTM(128, return_sequences = True),
                    layers.LSTM(128),
                    layers.Dense(prediction_size)])

    #early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience = 5, mode='min')
    model.compile(loss = "mse", 
              optimizer=Adam(learning_rate = 0.001),
              metrics=["mean_absolute_error"])
    
    checkpoint_filepath = 'best_model.h5'
    model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

    model.fit(X_train, y_train, validation_data = (X_val, y_val), epochs = 80, verbose = 0, callbacks = [model_checkpoint_callback], batch_size = 32)
    model = load_model(checkpoint_filepath)
    last_prices = data_scaled[-1*window_size:]
    last_prices_array = np.array(last_prices)
    input_pred = np.expand_dims(last_prices_array, axis=0)
    predictions = model.predict(input_pred)
    predictions = scaler.inverse_transform(predictions)
    array_predictions = [value for sublist in predictions for value in sublist]
    forecast_df = pd.DataFrame(array_predictions, columns=["Predicted Price"])
    return forecast_df

In [3]:
data = yf.download("MDO.DE", period = "max")
data.drop(columns = ["Open", "High", "Low", "Volume", "Adj Close"], axis = 1, inplace = True)
end_zeitpunkte = ["2023-07-07"]
end_zeitpunkte_ts = [datetime.strptime(end_zeitpunkte[0], "%Y-%m-%d").date()]

#hier Zeitraum der historischen Daten anpassen:
time_horizont = 3
start_zeitpunkte_ts = [end_zeitpunkte_ts[0]-relativedelta(years=time_horizont)]
filtered_df = data.loc[start_zeitpunkte_ts[0]:end_zeitpunkte_ts[0]]
hist_data = filtered_df[: len(filtered_df)-10]
hist_data_for_function = hist_data.copy()
prog_data = filtered_df[len(filtered_df)-10 :]
index_list = prog_data.index.tolist()

forecast_df = LSTM_OS_real(hist_data_for_function)

forecast_df.set_index(pd.Index(index_list), inplace=True)
result = pd.concat([prog_data, forecast_df], axis=1)

Note: 'info' dict is now fixed & improved, 'fast_info' no longer faster
[*********************100%***********************]  1 of 1 completed
1/1 [==============================] - 1s 834ms/step


In [4]:
result

,Close,Predicted Price
2023-06-26,263.899994,267.573242
2023-06-27,265.000000,267.396576
2023-06-28,266.500000,268.449036
2023-06-29,269.899994,267.041626
2023-06-30,272.000000,267.162933
2023-07-03,271.600006,266.868896
2023-07-04,271.299988,267.006470
2023-07-05,271.799988,266.553772
2023-07-06,270.700012,265.882141
2023-07-07,268.100006,266.066437


In [6]:
fig2 = px.line(template = "simple_white")
fig2.add_trace(go.Scatter(x = result.index.strftime("%Y-%m-%d"), y = result["Close"], mode = "lines", name = "Tatsächlicher Kurs", line_color = "red"))
fig2.add_trace(go.Scatter(x = result.index.strftime("%Y-%m-%d"), y = result["Predicted Price"], mode = "lines", name = "Prognose", line_color = "blue"))
fig2.update_layout(xaxis_title = "Datum", yaxis_title = "MC Donalds Kurspreis in EUR", xaxis= {"type": "category"}, legend=dict(x=0, y=1))
fig2.write_image("Prog_LSTM_OS.pdf")